In [18]:
import obspy

In [20]:
import glob
import os
from obspy import read


# Directory containing the files
directory_path = r'C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA'

# Glob all mseed files
mseed_files = glob.glob(os.path.join(directory_path, '*.mseed'))

# Open each mseed file using obspy
streams = []
for file in mseed_files:
    try:
        stream = read(file)
        streams.append(stream[0])
        print(f"Opened: {file}")
    except Exception as e:
        print(f"Error opening {file}: {e}")


Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-01-19HR00_evid00002.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-03-25HR00_evid00003.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-03-26HR00_evid00004.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-04-25HR00_evid00006.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-04-26HR00_evid00007.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-06-15HR00_evid00008.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-06-26HR00_evid00009.mseed
Opened: C:\Users\moaaz\Desktop\space_apps\data\lunar\training\data\S12_GradeA\xa.s12.00.mhz.1970-07-20HR00_evid00010.mseed
Opened: C:\Users

In [28]:
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    """
    Applies a bandpass Butterworth filter to the input data.

    Parameters:
    - data: Input signal (1D array)
    - lowcut: Low cutoff frequency for the bandpass filter in Hz
    - highcut: High cutoff frequency for the bandpass filter in Hz
    - fs: Sampling frequency in Hz
    - order: The order of the filter (default is 4)

    Returns:
    - filtered_data: Bandpass-filtered signal (1D array)
    """
    nyquist = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    print(low,high)
    # Design Butterworth bandpass filter
    b, a = butter(order, [low, high], btype='band')

    # Apply the filter using filtfilt to prevent phase distortion
    filtered_data = filtfilt(b, a, data)

    return filtered_data
def highpass_filter(data, cutoff, fs, order=4):
    """
    Applies a highpass Butterworth filter to the input data.

    Parameters:
    - data: Input signal (1D array)
    - cutoff: Cutoff frequency for the highpass filter in Hz
    - fs: Sampling frequency in Hz
    - order: The order of the filter (default is 4)

    Returns:
    - filtered_data: Highpass-filtered signal (1D array)
    """
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist

    # Design Butterworth highpass filter
    b, a = butter(order, normal_cutoff, btype='high')

    # Apply the filter using filtfilt to prevent phase distortion
    filtered_data = filtfilt(b, a, data)

    return filtered_data
def get_filtered(sig,highpass_cut=1, bandpass_lowcut= .1,bandpass_highcut =1.5, band_filt = True,high_filt=True):

    # Apply the high-pass filter to remove frequencies below 0.5 Hz
    filtered_trace = signal.detrend(sig)
    filtered_trace = signal.medfilt(filtered_trace, kernel_size=21)
    if high_filt:
        filtered_trace = highpass_filter(sig, highpass_cut, fs=20)
    if band_filt:
        filtered_trace = bandpass_filter(sig, bandpass_lowcut, bandpass_highcut,fs=20)


    return filtered_trace

In [22]:
stream = obspy.Stream(streams)

In [41]:
import scipy.signal as signal
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
stream[0].data
number_trace = 0
filtered_trace0 =  get_filtered(stream[number_trace].data,
                                #highpass_cut =.2,
                         #      bandpass_lowcut= .01,
                         #       bandpass_highcut =1,
                                band_filt=False,
                                )

In [59]:
for number_trace in range(0,len(stream)):
    frequencies, times, Sxx_0 = signal.spectrogram(stream[number_trace].data, stream[number_trace].stats.sampling_rate,
                                                nfft = int(stream[number_trace].stats.sampling_rate*120),
                                                nperseg=int(stream[number_trace].stats.sampling_rate*120) , 
                                                noverlap = int(stream[number_trace].stats.sampling_rate*30),
                                                scaling = 'density')
    fig, ax = plt.subplots(figsize=(12, 8),dpi=100)

    # Hide axes and spines
    ax.axis('off')
    # Plot the spectrogram
    pcm = ax.pcolormesh(times, frequencies, Sxx_0, cmap='viridis', shading='gouraud')
    plt.savefig(f'lunar_plot{number_trace}.png', dpi=100)
    plt.close(fig)
